In [5]:
import pandas as pd
import numpy as np
import states_abbrevs
from states_abbrevs import dict2
import glob

In [6]:
path = r"C:\Users\Steven Lantigua\Desktop\7ParkCP\data" 
files = glob.glob(path + "/*.csv")
years = ['2014','2015','2016','2017','2018','2019']
li=[]

for (f,y) in zip(files,years):
    df = pd.read_csv(f, index_col=None, header=0)
    df = df.loc[1:] #removes unnecessary column description
    df['Year'] = y
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
framec = frame.copy()

In [7]:
income = framec.loc[:,['Year','NAME','DP03_0062E']]
income.rename(columns={ 'NAME':'County', 'DP03_0062E':'Med_income'},inplace=True)

### fixing 'Location' format

In [8]:
income['county']=income['County'].str.split(',',1,expand=True)[0]
income['state'] = income['County'].str.split(',',1,expand=True)[1]
income['state'] = income['state'].apply(lambda x: x.strip())
income['state'] = income['state'].map(dict2)
income['County'] = income['county'] + ',' + income['state']
income.drop('county',axis=1,inplace=True)

 ### changing 'income' to integers

In [10]:
income['Med_income']=income['Med_income'].apply(lambda x: int(x))

### interpolating for month value

In [11]:
m_income = pd.DataFrame()
for county, df in income.groupby('County'):
    df['Change'] = df['Med_income'].diff() / df['Med_income'].shift()
    df = df.iloc[np.arange(df.shape[0]).repeat(12)].reset_index().drop('index', axis = 1)
    df = pd.concat([df, pd.DataFrame(list(range(1, 13)) * int(df.shape[0] / 12), columns = ['Month'])], axis = 1)
    df = df[df['Year'] != 2013]
    for year in pd.unique(df['Year']):
        df[df['Year'] == year]['Med_income'] = df[df['Year'] == 2014]['Med_income'] * (((1 + df[df['Year'] == 2014]['Change']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
    m_income = pd.concat([m_income, df], axis = 0)
    m_income.fillna(0,inplace=True)

<ipython-input-11-c48295fee3ca>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Med_income'] = df[df['Year'] == 2014]['Med_income'] * (((1 + df[df['Year'] == 2014]['Change']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])


In [12]:
income

,Year,County,Med_income,state
0,2014,"Penobscot County,ME",42242,ME
1,2014,"York County,ME",55509,ME
2,2014,"Allegany County,MD",39653,MD
3,2014,"Anne Arundel County,MD",87217,MD
4,2014,"Baltimore County,MD",68257,MD
...,...,...,...,...
4999,2019,"Ponce Municipio,PR",15130,PR
5000,2019,"San Juan Municipio,PR",23005,PR
5001,2019,"Toa Alta Municipio,PR",28440,PR
5002,2019,"Toa Baja Municipio,PR",23368,PR


In [13]:
m_income

,Year,County,Med_income,state,Change,Month
0,2014,"Ada County,ID",57716,ID,0.000000,1
1,2014,"Ada County,ID",57716,ID,0.000000,2
2,2014,"Ada County,ID",57716,ID,0.000000,3
3,2014,"Ada County,ID",57716,ID,0.000000,4
4,2014,"Ada County,ID",57716,ID,0.000000,5
...,...,...,...,...,...,...
67,2019,"Yuma County,AZ",46419,AZ,0.069488,8
68,2019,"Yuma County,AZ",46419,AZ,0.069488,9
69,2019,"Yuma County,AZ",46419,AZ,0.069488,10
70,2019,"Yuma County,AZ",46419,AZ,0.069488,11


In [14]:
m_income.to_csv('income.csv')